## Log-playback (expert) trajectory extraction and usage

This notebook demonstrates how to extract expert actions using different dynamics models and step through scene with those actions. 

In [1]:
import os
from pathlib import Path
import mediapy

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

from pygpudrive.env.config import EnvConfig, RenderConfig, SceneConfig
from pygpudrive.env.env_torch import GPUDriveTorchEnv

### Configurations

In [2]:
DYNAMICS_MODEL = "state" # "delta_local" or "state"
# DATA_PATH = "data/examples" # Your data path
DATA_PATH ="/mnt/nas_9/group/penghaocheng/code/multi_agent_with_LLM/dataset/output/visualization_data/test_small_scenes.txt"
MAX_NUM_OBJECTS = 128
NUM_ENVS = 10

# Configs
render_config = RenderConfig(draw_obj_idx=True)
scene_config = SceneConfig(path=DATA_PATH, num_scenes=NUM_ENVS)
env_config = EnvConfig(dynamics_model=DYNAMICS_MODEL)

### Make environment

In [3]:
env = GPUDriveTorchEnv(
    config=env_config,
    scene_config=scene_config,
    max_cont_agents=MAX_NUM_OBJECTS,
    device="cpu",
    render_config=render_config,
    action_type="continuous" # "continuous" or "discrete"
)

['/mnt/nas_9/group/penghaocheng/code/gpudrive/dataset/dataset/formatted_json_v2_no_tl_train_split_1000/batch_11/tfrecord-00829-of-01000_101.json', '/mnt/nas_9/group/penghaocheng/code/gpudrive/dataset/dataset/formatted_json_v2_no_tl_train_split_1000/batch_11/tfrecord-00879-of-01000_397.json', '/mnt/nas_9/group/penghaocheng/code/gpudrive/dataset/dataset/formatted_json_v2_no_tl_train_split_1000/batch_11/tfrecord-00931-of-01000_238.json', '/mnt/nas_9/group/penghaocheng/code/gpudrive/dataset/dataset/formatted_json_v2_no_tl_train_split_1000/batch_11/tfrecord-00951-of-01000_181.json', '/mnt/nas_9/group/penghaocheng/code/gpudrive/dataset/dataset/formatted_json_v2_no_tl_train_split_1000/batch_4/tfrecord-00079-of-01000_352.json', '/mnt/nas_9/group/penghaocheng/code/gpudrive/dataset/dataset/formatted_json_v2_no_tl_train_split_1000/batch_4/tfrecord-00105-of-01000_214.json', '/mnt/nas_9/group/penghaocheng/code/gpudrive/dataset/dataset/formatted_json_v2_no_tl_train_split_1000/batch_4/tfrecord-00129-

### Get log-playback (expert) actions

- Different dynamics models have different action spaces. For details, [see the docs.](https://github.com/Emerge-Lab/gpudrive/tree/main/pygpudrive/env)

In [4]:
obs = env.reset()

In [5]:
# Extract full expert trajectory
expert_actions, _, _ = env.get_expert_actions()

expert_actions.shape # Shape: (num_envs, num_steps, num_agents, num_actions)

torch.Size([10, 128, 91, 10])

### Step through an episode

In [6]:
# Reset environment
obs = env.reset()
import pickle
import numpy as np
mfile='/mnt/nas_9/group/penghaocheng/code/multi_agent_with_LLM/dataset/output/visualization_data/reserve_agent_mask_for_test_small.pkl'
with open(mfile,"rb") as m:
    mask=pickle.load(m)

frames = {f"env_{i}": [] for i in range(NUM_ENVS)}
path='/mnt/nas_9/group/penghaocheng/code/multi_agent_with_LLM/dataset/output/visualization_data/pred.pkl'
with open(path,"rb") as f:
    pred_trajs=pickle.load(f)

pred_trajs=np.transpose(pred_trajs,(0,2,1,3,4))
# print(pred_trajs.shape)#10,10,90,7,2
# Step through the scene
for t in range(env_config.episode_len):
    env.step_dynamics(expert_actions[:, :, t, :])
    
    # Render the scenes
    for i in range(NUM_ENVS):
        if i in [0,5,9]:
            frames[f"env_{i}"].append(env.render(i,t,pred_trajs[i],mask[i]))
        # if i ==0:
        #     break

### Show resulting trajectories!

In [7]:
# Show videos
import cv2
# os.makedirs('./videos',exist_ok=True)


for i in [0,5,9]:
    image_list=frames[f'env_{i}']
    os.makedirs(f'./results/env_{i}/',exist_ok=True)
    for j in range(0,len(image_list),9):
        cv2.imwrite(f'./results/env_{i}/{j}.jpg',image_list[j])
    # output_video = f"./videos/env_{i}.mp4"  # 输出视频文件名
    # fps = 10  # 设置帧率

    # # 获取图像尺寸
    # height, width, layers = image_list[0].shape

    # # 定义 VideoWriter，使用 mp4v 编码
    # fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # 适用于 .mp4
    # video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    # # 逐帧写入视频
    # for frame in image_list:
    #     video.write(frame)  # frame 已经是 NumPy 数组，不需要再读取

    # # 释放资源
    # video.release()
    # cv2.destroyAllWindows()

    # print("MP4 视频已保存:", output_video)
    # mediapy.show_videos(frames, fps=30, width=600, height=400, columns=2,codec='gif')